# find tracerRNA


In [1]:
# First created Thu Jul 11 09:51:15 CEST 2019
# Made by L-F-S
# At the University Of Trento, Italy

import os
import sys
import subprocess
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.Alphabet import generic_dna, generic_protein, RNAAlphabet
from Bio import pairwise2


sys.path.insert(0, '/home/lorenzo.signorini/cas_mining/utils/')
import filename_discrepancies
import originalpath
sys.path.insert(0, '/home/lorenzo.signorini/cas_mining/9casminer_v2/')
import locus

feature="Cas9" #WARNING!!! CHANGE THIS!!
#outpath="/shares/CIBIO-Storage/CM/scratch/tmp_projects/signorini_cas/5caslocitable/out/"+feature+"/"
datadir="/shares/CIBIO-Storage/CM/news/users/lorenzo.signorini/7tracrRNA/"
outdir="/shares/CIBIO-Storage/CM/news/users/lorenzo.signorini/9output/"+feature+"/"



In [2]:
# Import data
cas_dataset=pd.read_csv("/shares/CIBIO-Storage/CM/news/users/lorenzo.signorini/5caslocitable/known_"+feature+"_variants_table.csv", index_col=0)
cas_dataset.columns

Index(['Seq ID', 'Seq Description', 'Seq', 'Contig', 'Genome Name', 'Study',
       'Sample Name', 'SGB ID', 'Pos', 'pilercr_CRISPR', 'minced_CRISPR',
       'prokka_cas1', 'prokka_cas2', 'uSGB', 'Level of estimated taxonomy',
       'Estimated taxonomy', 'prokka_cas9'],
      dtype='object')

In [14]:
#seqids: AOPFDEBE_00006
# FBDDNBEO_00645
# HPLHKOKB_00548
# LPEAHEHG_01071
# CJFLGLGO_01367
# AJHGDLDC_00764
# HPLHKOKB_00548
# LEAPHKCP_01380
# DOOEBCFP_00186

In [3]:
seqid ="FBDDNBEO_00645"   # MANUAL INPUT
outdir=outdir+seqid+"/"
if not os.path.exists(outdir):
    os.makedirs(outdir)
print("SEQ ID: ", seqid)
print(cas_dataset[cas_dataset["Seq ID"]==seqid]["Contig"].iloc[0])
print(cas_dataset[cas_dataset["Seq ID"]==seqid]["Study"].iloc[0])
print(cas_dataset[cas_dataset["Seq ID"]==seqid]["Genome Name"].iloc[0])
print(cas_dataset[cas_dataset["Seq ID"]==seqid]["SGB ID"].iloc[0])
print(len(cas_dataset[cas_dataset["Seq ID"]==seqid]["Seq"].iloc[0]))
print(cas_dataset[cas_dataset["Seq ID"]==seqid]["minced_CRISPR"].iloc[0])
print("----------------------------------------------------------------------")


SEQ ID:  FBDDNBEO_00645
NODE_2092_length_11346_cov_6.0000_ID_47560
ZeeviD_2015
ZeeviD_2015__PNP_Main_471__bin.13
8770
1034
[('.', '27', '260')]
----------------------------------------------------------------------


## check there is only 1 Cas9 in that genome..

In [4]:
genomename=cas_dataset[cas_dataset["Seq ID"]==seqid]["Genome Name"].iloc[0]
cas_dataset[cas_dataset["Genome Name"]==genomename]

,Seq ID,Seq Description,Seq,Contig,Genome Name,Study,Sample Name,SGB ID,Pos,pilercr_CRISPR,minced_CRISPR,prokka_cas1,prokka_cas2,uSGB,Level of estimated taxonomy,Estimated taxonomy,prokka_cas9
30968,FBDDNBEO_00645,FBDDNBEO_00645 CRISPR-associated endonuclease ...,MGAEFRHVTSLLLDEDFGEIKTAAGLRRQMRTRKAHKAREKWLERC...,NODE_2092_length_11346_cov_6.0000_ID_47560,ZeeviD_2015__PNP_Main_471__bin.13,ZeeviD_2015,PNP_Main_471,8770,1663 4767,"[('ID=CRISPR1', '.', '4')]","[('.', '27', '260')]","[('ID=FBDDNBEO_00644', '776', '1666')]","[('ID=FBDDNBEO_00643', '378', '728')]",Yes,Other,-,"('ID=FBDDNBEO_00645', '1663', '4767')"


In [19]:
cas9_aa=cas_dataset[cas_dataset["Seq ID"]==seqid]["Seq"].iloc[0]
dataset=cas_dataset[cas_dataset["Seq ID"]==seqid]["Study"].iloc[0]
genomename=cas_dataset[cas_dataset["Seq ID"]==seqid]["Genome Name"].iloc[0]
SGB=cas_dataset[cas_dataset["Seq ID"]==seqid]["SGB ID"].iloc[0]
contigname=cas_dataset[cas_dataset["Seq ID"]==seqid]["Contig"].iloc[0]
samplename = genomename.split("__")[1]

print("\n-----------------------------------------------------\n"+feature+" id:\t"+seqid+"\nSGB:\t"+str(SGB)+"\nGenome Name:\t",\
      genomename+"\nContig:\t"+cas_dataset[cas_dataset["Seq ID"]==seqid]["Contig"].iloc[0],"\n-----------------------------------------------------\n")

print("..working genomename:", filename_discrepancies.get_originalsamplename_froms3name_of_genome(genomename,dataset))
########################################################################

# get CRISPR spacer and repeat sequence

cr=locus.CRISPRarray(feature=feature, contigname=contigname, genomename=genomename, datasetname=dataset)
cr.get_CRISPR_array()
spacers=cr.spacers
repeats=cr.repeats
repeat_start_pos=cr.repstartpos
unique_repeats=np.unique(repeats)
# print repeats

CRISPRheader="REPEAT"+" "*(len(cr.repeats[0])-len("REPEAT"))+" SPACER"+" "*(len(cr.spacers)-len("SPACERS"))+"\n"
print("CRISPR array sequence for "+ seqid+" "+feature+".")                                         
print(CRISPRheader)                                                     
CRISPRarraysequence=""                                                  
for i in range(len(cr.repeats)):                      
    try:                                                                
        cr.spacers[i]                                 
        line=cr.repeats[i]+" "+cr.spacers[i]
        print(line)                                                     
        CRISPRarraysequence+=line+"\n"                                  
    except:                                                             
        break
                                                                        #

print("unique repeats:")
for i in range(len(unique_repeats)):
    print("repeat ",str(i+1),":\t",unique_repeats[i])
print("cp "+cr.path+" "+datadir)


# get whole locus sequence todo this could become a function of its own, dentro utils, gli dai seqid della cas e ti dà tt il locus

# > get start and stop position of all Cass. questo devi metterlo nella tabella di cas non ci sono santi.
cas_position={feature:[int(n) for n in cas_dataset[cas_dataset["Seq ID"]==seqid]["Pos"].iloc[0].split()],\
              "Cas1":[int(n) for n in eval(cas_dataset[cas_dataset["Seq ID"]==seqid]["prokka_cas1"].iloc[0])[0][1:]],\
              "Cas2":[int(n) for n in eval(cas_dataset[cas_dataset["Seq ID"]==seqid]["prokka_cas2"].iloc[0])[0][1:]],\
              "CRISPR":[int(n) for n in eval(cas_dataset[cas_dataset["Seq ID"]==seqid]["minced_CRISPR"].iloc[0])[0][1:]]
             }
print("Locus features and positions: ", cas_position)
# so già che all casses inside here are on the same contig which is 'contig', xke le ho estratte per essere così
# FALSO! è cosi in cas_dataset, non in tabellazzza! mi ero confuso! trovato il bug!

full_genome_full_path, annotation_folder=originalpath.print_path(genomename)

print("+"*80)
print(cas_position)
print("Retrieving locus")
#print("Copying file ot output...")
#print("cp "+genomefilename+" "+datadir)

# access contig:

for record in SeqIO.parse(full_genome_full_path,"fasta"):  #c'è modo di non fare questo 'ciclo'?
    if record.id.startswith(contigname):
        #test se effettivamente la seq di cas è dove voglio io

        SeqIO.write(record, datadir+"tempblastdb", "fasta")
        print("CRISPR si trova", cas_position["CRISPR"])
        print("Cas9 si trova", cas_position[feature])
        print("Cas2 si trova", cas_position["Cas2"])
        print("Cas1 si trova", cas_position["Cas1"])

        #estrai la posizione di cas9 dal contig, e traducila
        cas9start=cas_position[feature][0]
        cas9stop=cas_position[feature][1]
        cas9_nnseq=record.seq[cas9start-1:cas9stop-3] #gff should have a 1-based positional annotation (ma sto andando a occhio finchè non sono uguali, è già un oggetto Bio.Seq

        my_translated_cas9=cas9_nnseq.transcribe().translate()

  
        print(len(cas9_nnseq)/3,len(cas9_aa))


        if my_translated_cas9==cas9_aa:
            print("Cas locus on plus strand")
            plus=True
        if not my_translated_cas9==cas9_aa:
            #proviamo col reverse complement
            cas9_nnseq=record.seq[cas9start+2:cas9stop]
            my_translated_cas9=cas9_nnseq.reverse_complement().transcribe().translate()
            print("Cas locus on minus strand")
        print("Is Cas9 translated from the genome equal to the orignal annotation?", my_translated_cas9==cas9_aa)
        print(my_translated_cas9)
        print("+"*80)
        print(cas9_aa)
        
        alignments = pairwise2.align.localxx(my_translated_cas9,cas9_aa)
        print(pairwise2.format_alignment(*alignments[0]))
        
        #save .faa aminoacid sequence to output: (commented out as not to overlap with get_ID_info)
      #  if my_translated_cas9==cas9_aa:
       #     print("SAVING aminoacid sequence to file in "+outdir+"...")
        #    cas9aafilename=str(SGB)+"__"+seqid+"__"+genomename+"."+feature+".faa"
         #   f=open(outdir+cas9aafilename,"w") #TODO dir
          #  f.write(">"+seqid+"\n"+cas9_aa)
           # f.close()

        #opzione 1 blast
      #1. salva np.uniq(repeats) in un fasta file: build temporary query file for blastn search

        print("let's now blast the repeat against  the genome")
        os.chdir(datadir)
        tempfile=open("temp_repeat_seq", "w")
        tempfile.close()
        tempfile=open("temp_repeat_seq", "a")

        for n, repeat in enumerate([Seq(sequence) for sequence in np.unique(repeats)]):
            tempfile.write(">rpt"+str(n+1)+"|"+contigname+"|"+genomename+"|"+seqid+"\n"+str(repeat)+"\n") 
        tempfile.close()

        #2. Blast query file against db

        #3. make db file
        print("Build blast database with:")
        dbfile=datadir+"tempblastdb"
        print("/home/lorenzo.signorini/ncbi-blast-2.10.0+/bin/makeblastdb -in "+dbfile+" -parse_seqids  -dbtype nucl")
        os.popen("makeblastdb -in "+dbfile+" -parse_seqids  -dbtype nucl")


        print("Run blastn of query file agianst all contigs")
        blastoutfile=datadir+"temp.blastout"
        blastn_command = "/home/lorenzo.signorini/ncbi-blast-2.10.0+/bin/blastn -out "+blastoutfile+" -outfmt \"6  qseqid sseqid pident qlen length mismatch gapopen qseq sseq sstart send evalue sstrand\" -query temp_repeat_seq -db "+dbfile+" -evalue 0.001 -word_size 11 -penalty -2"  #MANUAL INPUT
        print(blastn_command)
        os.popen(blastn_command)
       # os.chdir(datadir)
      #  subprocess.call(blastn_command) does not work



-----------------------------------------------------
Cas9 id:	LPEAHEHG_01071
SGB:	15299
Genome Name:	 ChengpingW_2017__AS43raw__bin.51
Contig:	NODE_190_length_65757_cov_25.3876 
-----------------------------------------------------

..working genomename: ('ChengpingW_2017__AS43raw__bin.51', 'ChengpingW_2017')
CRISPR array sequence for LPEAHEHG_01071 Cas9.
REPEAT                               SPACER

GGGATGATTTACTAATAGCTCTTGGTATGGTATAAT CTTCCGTGCCATTTCCTCTATGACAGAATC
GTCATAGTTCCCTAACAGCTCTTGGTATGGTATAAT CTGCCCGGCGACGTGCGTGTGAGCGATACG
GTCATAGTTCCCTAATAGCTCTTGGTATGGTATAAT CCTGCCGCCCAATGCCCAGGACAACGAACT
GTCATAGTTCCCTAATAGCTCTTGGTATGGTATAAT AACGACTATTCAACGCACGGTAATAGCGCCT
unique repeats:
repeat  1 :	 GGGATGATTTACTAATAGCTCTTGGTATGGTATAAT
repeat  2 :	 GTCATAGTTCCCTAACAGCTCTTGGTATGGTATAAT
repeat  3 :	 GTCATAGTTCCCTAATAGCTCTTGGTATGGTATAAT
cp /shares/CIBIO-Storage/CM/news/users/lorenzo.signorini/1crisprsearch/out/ChengpingW_2017/ChengpingW_2017__AS43raw__bin.51.fa.minced.out /shares/CIBIO-Stor

In [9]:
#4. parse blast output: only print output if it is NOT one of the old repeats.
print("\n-----------------------------------------------------\nParsing BLAST output, looking for hit which are NOT the same repeats\ni.e. not starting with:", repeat_start_pos,":\n-----------------------------------------------------\n")
f=open(blastoutfile,"r")
for line in f.readlines():
    start_pos=line.split("\t")[9]
    if start_pos not in repeat_start_pos:
        if str(int(start_pos)+1) not in repeat_start_pos:
            if str(int(start_pos)-1) not in repeat_start_pos:
                print(line) 


-----------------------------------------------------
Parsing BLAST output, looking for hit which are NOT the same repeats
i.e. not starting with: ['9802', '9868', '9934', '10000', '10067'] :
-----------------------------------------------------



NameError: name 'blastoutfile' is not defined

## extract sequences

In [9]:
# manual input
# COSA NON SO A PRIORI: l'orientamento del CRISPRarray (e quindi dell'antirepeat)
# COSA SO A PRIORI: una cosa sullo strand + si legge SEMPRE E SOLO da SINISTRA VERSO DESTRA
#     una cosa sullo strand - si legge SEMPRE E SOLO da DESTRA VERSO SINISTRA
# il DNA, come molecola, è polarizzato. LE basi sono come un parcheggio a spina di pesce,
# con un angolo e
# quindi una direzione.
# COSA DEVO TROVARE: il match della repeat (LA SECONDA FRA LE DUE SEQUENZE NEL BLAST), e un terminatore nelle sue vicinanze. 
# SE è alla sua DESTRA
# devo prendere il filamento sullo strand + 
# (che potrebbe essere SIA la repeat che ha matchato il BLAST
# direttamente SIA il suo reverse complement. NOn lo so prima xke nn so dove stia
# (== su che strand e in che direzuione sia scritto) CRISPR, lo
# indurrò a posteriori)
# ; SE è  a SINISTRA,
# devo prendere il filamento sullo strand -.
#
# devo, dunque, SCEGLIERE se l'antirepeat è la matched repeat o il suo rev complement 
# a seconda di DOVE STA IL TERMINATORE
# così facendo, identificherò anche l'orinetamento del CRISPR
#
#ESEMPIO
#  se .la matched repeat  è sullos trand - VUOL DIRE CHE VA LETTA DA DESTRA VERSO SINISTRA
# e bisogna vedere dove sara il terminator:
# se è A DESTRA della matched e sullo strnaad + (== si legge da sinistra)
# , allora l'antirepeat sarà effettivamente il reverse complement della matched repeat
#, se  è  A SINISTRA della matched e sullo strand -(==si legge da destra), allora
# il mio antirepeat è in effetti la matched_repeat, e quindi probabilmente il CRISPRarray( essendo palindromo)
# è stato annotato sullo strand opposto. 
# Se il terminatore è A DESTRA della matched (sempre nel caso matched  su -)
# e sullo strand - allora è un terminatore SBAGLIATO
#
# FILL IN INPUTS MANUALLY in the lines with the 'manual input' comment!
mstrand=-1  #manual input: strand of matched repeat
matched_rep_start=8422  #manual input ALWAYS input the smaller number here, even if ti matches on - strand
matched_rep_stop=8456   #manual input



In [10]:
##### other seqids
##    seqid   #    mached_rep_start   # matched _rep_stop
#AOPFDEBE_00006          6358         6376
#HPLHKOKB_00548          1514         1541
#FBDDNBEO_00645         297            318
#LPEAHEHG_01071         14729         14751            +1
#CJFLGLGO_01367         49347         49380            +1    
# HPLHKOKB_00548        1541        1514                -1
# LEAPHKCP_01380       

In [11]:
unique_repeats

array(['ATTATATCGCATTTTTGTTGGAAAGGAAGCGATAAC'], dtype='<U36')

In [12]:
# this is the sequence MATCHING the repeat (NOT the repeat itself):
matched_rep=Seq("ATTATTTGCGTTTTTGTTGGAAAGGAAGCCGATAA")  #manual input (remove gaps if present)
# siccome non mi interessa per ora sapere il verso del CRISPRarray, facciamo che la matchedrepeat e il su revocm
# li chiamo matched_rpt_fwd e matched_rpt_rev:

matched_rpt_fwd=matched_rep if mstrand == 1 else  matched_rep.reverse_complement()
matched_rpt_rev=matched_rep if mstrand== -1 else matched_rep.reverse_complement()

#rpts which match:
rpt=Seq(unique_repeats[0])  #manual input #TODO unique_repeats non e' una lista ordinata:la posizione non e' per forza quella giusta
initial_repeat=rpt

#this should go frombo to bo  = x nt  (x+1)
#`fwd and rvrs are referred to the strand.
print("matched rep fwd:", matched_rpt_fwd,"matched rep rev: ", matched_rpt_rev)
len(matched_rpt_fwd)



matched rep fwd: TTATCGGCTTCCTTTCCAACAAAAACGCAAATAAT matched rep rev:  ATTATTTGCGTTTTTGTTGGAAAGGAAGCCGATAA


35

In [13]:
str(SGB)+"__"+seqid+"__"+genomename+".terminators"

'9311__LEAPHKCP_01380__BritoIL_2016__M2.36.ST__bin.48.terminators'

In [14]:
for record in SeqIO.parse(full_genome_full_path, "fasta"):
    if record.id==contigname:
        contigseq=record.seq
        print(contigseq)

GATCTGACGATGAAATCGCTGTCAGCCGTCAGCATTATCCAATACGACACGGCGCTGAAATCGCTGATTGAATACGGACGCACGATTAACTTGGGCGTCGTGGCTTTGATGAAAAACGGCGATTTGCAGATCCCCCGCGACCCTCAGCTGAAGCCCGAAGCCCGTCGCGCTTTGGCGGTTGTTATCGGGTCGGATACGGGGCTGGTCGGCAAAATGAACCGCGAAGTTGTGCAGTACGCGCAGGATTTTCTGCGTCAAAAGGGCTTTACGCTGGACCAAGTGAACTTTATCGCCGTCGGACGTCAAATCATCGGTCAGCTGATGCGCGGCAAGCAAAACGTCATTGAAGCCTATCCTGTTTCAAATTCGGTCAAGGCGGTCAGCAAAACGGCGGCTTCCGTTCTGATTACGGCGGACAAATACATTCATACGCACCACGTGACGCACGTTTACGTTTTTTACAGCGAAAAACGCGGCGTCAGCGTGTCCGCGACGGAAACGTTGCTGATGCCGCTGGGAACGGAGTGGATGGAACGGCTGAAAAAACGCGGATGGCCCGGACGCAATCTGCCGACGTACACTTTGCCGCCCGAAAAGATTTACTCCGCCCTGCTCAAAGAGCGCTTGCTGATTGAACTGTCCACCGCGATAACGGAAGCGTTGTCCGCCGAGCATCACATGCGTTTGACGACGATGCAGGCGGCGGAAAAGAACATTGACGAAAACTTGGAAGCGATGAACTTGCAGTATCAGCAGATGCGGCAGGAAAACATCACGACCGAACTGCTGGACGTCGTCAACGGCGCCGAAGCAATGAGAAAGAAAAAGTGATGCCGCGCTTGATTCGGGAAATCCGCCGCAGGACGCGCCATGTTTTGCCGCCTTTGTTTTGGCTGGTGCTGACATTTTATTTCGGATGGCACGCAGTCAACGGCGAACGCGGCCTGCGCCGCATGTTTGAACTGAAGCAGGAAATCGCGATTGCCAAGCAAGTCGCCGA

In [15]:
# print terminators

f=open(datadir+str(SGB)+"__"+seqid+"__"+genomename+".terminators") 
for line in f.readlines():
    if line.strip().startswith(""): #manual input
        print(line)
f.close()

   1177  Rnamotif - GCGGGCATTTACGCGGCAATCGCCGTGTTTTTGACGTTC -10.00

   2355      Both - GATATTTTCGTTGGCGACGGACGCCATTTTTCTGAACG -6.90

   3685      Both - TTTGTTTGAAAAACGGCAACCACAGCCTTGCCGTTTTGCGATTTTCAT -10.10

   3689      Both + AAATCGCAAAACGGCAAGGCTGTGGTTGCCGTTTTTCAAACAA -8.80

   4750     Erpin - CACATAAAAAACAGGCGAAGGTTAAAAACTTTCGCCTGTTTTGTTTTATCG -11.60

   4752      Both + ATAAAACAAAACAGGCGAAAGTTTTTAACCTTCGCCTGTTTTTTATGTGTT -11.60

   4753      Both - CATAAAAAACAGGCGAAGGTTAAAAACTTTCGCCTGTTTTGTTTTA -11.20

   4811      Both + GGTAAAAAAGTGCCGCCGCCTTTAGGGGAAAAGGCGGCGGCTTTCAGATTCCTT -16.90

   5467  Rnamotif + ACGGTATGCAATGCCCGCGCGGGTGTTTTTGGTGAAT -4.80

   5760  Rnamotif - AGCGACCGCGTTGCCGTCGGTTTCGGCGTTTATGTTCCGT -6.20

   6941  Rnamotif + GACGGTTATTGCGCCGACGGGGCAGGCGaTTTTACACGCTC -6.30

   7846      Both - AAATACGTTCTTGCCGCTTTGGTGTTCGGCGTTTTAATCGGCG -4.60

   8305     Erpin - CCTCAGAAGAAAAGGGGGCTGACCGCGCGTCACCCCCTTTTCATTTTGCCGT NA

   8307      Both + GGCAAAATGAAAAGGGGGTGACGCGCGGTC

In [21]:
#FBDDNBEO_00645  319
#LPEAHEHG_01071 14653
#CJFLGLGO_01367 49290
#Aadadssada            6865
# HPLHKOKB_00548  1649


In [16]:
#29/01/2020
def get_real_repeat(rpt, crisprstrand):
    return rpt if crisprstrand == 1 else rpt.reverse_complement()
#29/01/2020 final fix

tstrand=-1  #manual input, terminator strand. TODO CHECK che se lo strand è -1 allora è <matched rep (tracrRNA on - strand),e viceversa

terminator_start=8305 #manual input N.b. this is always the smallest position (between start and end)
terminator=Seq("CCTCAGAAGAAAAGGGGGCTGACCGCGCGTCACCCCCTTTTCATTTTGCCGT".upper()) #manual input
terminator_end=terminator_start+len(terminator)
if tstrand==1:  # if terminator on + strand
    if matched_rep_stop<terminator_start: # check that term is on the RIGHT SIDE of the repeat
        print("Strand OK")
        antirepeat=matched_rpt_fwd
        print("Is antirepeat the reverse complement of original repeat? (WARNING: original repeat is just the initial annotation, may not be the correct orientation of the repeat!)", True if mstrand == -1 else False)        
        tracrstrand=tstrand
        print("antirepeat: ",antirepeat)
        crisprstrand =  -mstrand #-mstrand if mstrand == tstrand else -mstrand
        rpt=get_real_repeat(rpt, crisprstrand)
    else:
        print("STRAND NOT OK", matched_rep_start, matched_rep_stop)
elif tstrand==-1:
    if matched_rep_start>terminator_start: # check that term is on the LEFT SIDE of the repeat
        print("Strand OK")
        antirepeat=matched_rpt_rev
        tracrstrand=tstrand
        #todo rpt= ?
        print("antirepeat: ",antirepeat)
        crisprstrand = mstrand # cioé: mstrand if mstrand == tstrand else mstrand
        rpt=get_real_repeat(rpt, crisprstrand)
        print("Is antirepeat the reverse complement of original repeat? (WARNING: original repeat is just the initial annotation, may not be the correct orientation of the repeat!)", False if mstrand == -1 else True)        
    else:
        print("STRAND NOT OK", matched_rep_start, matched_rep_stop)
else:
    print("input t strand please")


print("terminator length: ", len(terminator))
print("terminator start and stop:",terminator_start, terminator_end)
#print("nb start and stop are just the position from smallest to biggest, without direction. Pay attention when working on minus strand")
print("tracr strand=",tracrstrand)
print("CRISPR strand=", crisprstrand)
print("Repeat:", rpt,"\t(original repeat:\t", initial_repeat+")")

Strand OK
antirepeat:  ATTATTTGCGTTTTTGTTGGAAAGGAAGCCGATAA
Is antirepeat the reverse complement of original repeat? (WARNING: original repeat is just the initial annotation, may not be the correct orientation of the repeat!) False
terminator length:  52
terminator start and stop: 8305 8357
tracr strand= -1
CRISPR strand= -1
Repeat: GTTATCGCTTCCTTTCCAACAAAAATGCGATATAAT 	(original repeat:	 ATTATATCGCATTTTTGTTGGAAAGGAAGCGATAAC)


str(rpt)

In [25]:
print("forward, 150 nt",contigseq[matched_rep_start-1:matched_rep_start+149])
print("reverese, 150nt", contigseq[matched_rep_stop-150:matched_rep_stop].reverse_complement())

forward, 150 nt TTATAGACTTCCTTTCCAAATCAGACATAAAATAACAAAAGTTCTCTGATTAAGAGAATATTACGGAATGCTGTTTCCGTCAAGTGGAATTTTTACTTTCGGAATAAGACGCCTGAGATTGCAGTTTTCCCGGACAGTGAAAGAAAAAGC
reverese, 150nt ATGTCTGATTTGGAAAGGAAGTCTATAATAATCGAAGTTTTCTTTACGAGTAGGGCTCTGACGTCTCATATAATATATGAGGCGTCATCCTTTAAATAATTGTCCCACAGCAAGTTGCAGTTTCTCAAGCAGCAGCAAAGTTAATGTTTT


In [64]:
# Open genome and extract the whole sequence
# find this inside the genome, and link it to the thing found by ARNold:

#I need to specify BOTH the strand and the direction
#if strand is +, then the actual antirepeat is going to be what i called antirepeat 
#only if the repeat blast matched on the minus strand (vedi schema), e vari ncastri complciati"    # temP: matched reè start -1:matched rep stop

for record in SeqIO.parse(full_genome_full_path, "fasta"):
    if record.id==contigname:
        contigseq=record.seq

#check antirepeat
extracted_antirepeat =  contigseq[matched_rep_start-1:matched_rep_stop] if tracrstrand==1 else contigseq[matched_rep_start-1:matched_rep_stop].reverse_complement()
print("test antirepeat: ", antirepeat, len(antirepeat), extracted_antirepeat==antirepeat) # -1 why? Indexing diverso evidentemente col blastout, vbb

# check terminator
extracted_terminator =  contigseq[terminator_start-1:terminator_end-1] if tracrstrand==1 else contigseq[terminator_start-1:terminator_end-1].reverse_complement()
print("test terminator: " ,terminator, len(terminator), extracted_terminator==terminator)


pre_tracrRNA= contigseq[matched_rep_start:terminator_end] if tracrstrand==1 else contigseq[terminator_start:matched_rep_stop].reverse_complement()
print("pre tracrRNA:", pre_tracrRNA, len(pre_tracrRNA))

# add mario
mario= contigseq[matched_rep_stop:terminator_start-1] if tracrstrand==1 else contigseq[terminator_end:matched_rep_start-1].reverse_complement()
print("mario sequence:",  mario, len(mario))  # ho fatto i check, il -1 sopra è giusto

test antirepeat:  ATTATTTGCGTTTTTGTTGGAAAGGAAGCCGATAA 35 True
test terminator:  CCTCAGAAGAAAAGGGGGCTGACCGCGCGTCACCCCCTTTTCATTTTGCCGT 52 True
pre tracrRNA: ATTATTTGCGTTTTTGTTGGAAAGGAAGCCGATAATAATCGAAACGAAAGTTTTCCGAGTGGGGCAACGGCGATCCGGTCTTGTATCGGGGTAGCCGTCCCCTCAGAAGAAAAGGGGGCTGACCGCGCGTCACCCCCTTTTCATTTTGCCG 151
mario sequence: TAATCGAAACGAAAGTTTTCCGAGTGGGGCAACGGCGATCCGGTCTTGTATCGGGGTAGCCGTC 64


In [65]:
print(pre_tracrRNA.transcribe())

AUUAUUUGCGUUUUUGUUGGAAAGGAAGCCGAUAAUAAUCGAAACGAAAGUUUUCCGAGUGGGGCAACGGCGAUCCGGUCUUGUAUCGGGGUAGCCGUCCCCUCAGAAGAAAAGGGGGCUGACCGCGCGUCACCCCCUUUUCAUUUUGCCG


In [66]:
print(extracted_antirepeat.transcribe())

AUUAUUUGCGUUUUUGUUGGAAAGGAAGCCGAUAA


In [67]:
print(terminator_end, matched_rep_stop)

8357 8456


In [68]:
# Now, align with the rest of the stuff to see what's up
# first up, with the repeat, lets take the rpt3, which has only got 1 mismatch
# x AOPFDEBE: extracted antirepeat: matched_stop+11

alignments = pairwise2.align.localxx(rpt, extracted_antirepeat.reverse_complement())  #29/01/2020 fixed
print(pairwise2.format_alignment(*alignments[0]))
print("rpt length:", len(rpt))


GTTATCG-CTTCCTTTCCAACAAAAATGCG-ATA-TAAT
 |||||| ||||||||||||||||||  || | | ||||
-TTATCGGCTTCCTTTCCAACAAAAA--CGCA-AATAAT
  Score=32

rpt length: 36


In [69]:
# align antirepat + mario with repeat
alignments = pairwise2.align.globalxx( rpt,(antirepeat+mario[:4]).reverse_complement())
print(pairwise2.format_alignment(*alignments[0]))

G-TTAT---CG-CTTCCTTTCCAACAAAAATGCG-ATA-TAAT
  ||||   || ||||||||||||||||||  || | | ||||
-ATTATTATCGGCTTCCTTTCCAACAAAAA--CGCA-AATAAT
  Score=32



In [70]:
# I would probably say that the tracr is just, simply la parte di pre_tracrRNA la cui antirepeat si lega alla repeat, 
# che, in questo caso, è tutta.
tracrRNA = pre_tracrRNA

line="\n----------------------------------------------------------------------"+"\n"+\
"         tracrRNA sequence for "+ feature+" protein id "+ seqid+"\n"+\
"----------------------------------------------------------------------\n"+\
"- contig name: "+ cas_dataset[cas_dataset["Seq ID"]==seqid]["Contig"].iloc[0]+"\n"+\
"- Dataset: "+cas_dataset[cas_dataset["Seq ID"]==seqid]["Study"].iloc[0]+"\n"+\
"- Bin: "+ cas_dataset[cas_dataset["Seq ID"]==seqid]["Genome Name"].iloc[0]+"\n"+\
"- SGB: "+ str(cas_dataset[cas_dataset["Seq ID"]==seqid]["SGB ID"].iloc[0])+"\n"+\
"----------------------------------------------------------------------\n"+\
"- Length: "+ str(len(tracrRNA))+"\n- position: "+ str(terminator_end)+" "+str(matched_rep_stop)+"\n"+\
"- Sequence: \n\t"+str(tracrRNA.transcribe())+"\n"+\
"- tracrRNA strand: "+ str(tracrstrand)+"\n"+\
"- CRISPRarray real strand: "+ str(crisprstrand)+"\n"+\
" - chosen repeat:\n"+ str(rpt) 
print(line)




----------------------------------------------------------------------
         tracrRNA sequence for Cas9 protein id LEAPHKCP_01380
----------------------------------------------------------------------
- contig name: NODE_1601_length_16622_cov_6.41836
- Dataset: BritoIL_2016
- Bin: BritoIL_2016__M2.36.ST__bin.48
- SGB: 9311
----------------------------------------------------------------------
- Length: 151
- position: 8357 8456
- Sequence: 
	AUUAUUUGCGUUUUUGUUGGAAAGGAAGCCGAUAAUAAUCGAAACGAAAGUUUUCCGAGUGGGGCAACGGCGAUCCGGUCUUGUAUCGGGGUAGCCGUCCCCUCAGAAGAAAAGGGGGCUGACCGCGCGUCACCCCCUUUUCAUUUUGCCG
- tracrRNA strand: -1
- CRISPRarray real strand: -1
 - chosen repeat:
GTTATCGCTTCCTTTCCAACAAAAATGCGATATAAT


In [71]:
# Write output to file
#if not os.path.exists(outdir):
 #   os.makedirs(outdir)
#f=open(outdir+"/"+"tracrRNA_"+seqid+".txt","w")
#f.write(line)
#f.close()



In [72]:
print("To save all output  using get_ID_info (recomended)")
print("get_ID_info "+seqid+" "+"-t "+str(tracrRNA.transcribe())+" -s "+str(tracrstrand)+" -c "+str(crisprstrand)+" -r "+str(rpt)+" -v -d")

To save all output  using get_ID_info (recomended)
get_ID_info LEAPHKCP_01380 -t AUUAUUUGCGUUUUUGUUGGAAAGGAAGCCGAUAAUAAUCGAAACGAAAGUUUUCCGAGUGGGGCAACGGCGAUCCGGUCUUGUAUCGGGGUAGCCGUCCCCUCAGAAGAAAAGGGGGCUGACCGCGCGUCACCCCCUUUUCAUUUUGCCG -s -1 -c -1 -r GTTATCGCTTCCTTTCCAACAAAAATGCGATATAAT -v -d


In [ ]:
# AOPFDEBE_00006:
get_ID_info AOPFDEBE_00006 -v -d -t GACGAUUUGAGAAGCAACCGUUUAUAUAAUAAAGAAUAUUUUCUGUGGGGUUCUACGAUUAUGUAUCCCCUGAAACCGUCCCCCUGCGGGGACGGUUUCUGUUGUAU -s - -c -1  -r GTTATAGGTTGCTTCTCAAATCGTCTTTGGTATAAT
# CJFLGLGO_01367:
get_ID_info CJFLGLGO_01367 -t UUAUAACAUACCAAGAACAAUUGGGUUACUACAAAAAGGUAGAAAACCGAAAAGCUCUGACGGCUCCUUUUUGGAGCCGUUAUCUUUUUC -s -1 -c 1 -r ATTGTAGTTCCCTAATTTTTCTTGGTATGTTATAAT -v -d
# AJHGDLDC_00764:
get_ID_info AJHGDLDC_00764 -t UUAUACCAUACCAAGAACUAUUAUGGUUGCUAUGAUAAGGUCAUAGGACCGUAAAGCUCUGACGCCCUGCUAUUUGGCAGGGCGUCAUCUUUUUUGCC -s -1 -c 1 -r ATCATAGTTCCCTAATAGCTCTTGGTATGGTATAAT -v -d
# LPEAHEHG_01071:
get_ID_info LPEAHEHG_01071 -t UUAUACCAUACCAAGAACUAUUAUGGUUGCUAUGAUAAGGUCAUAGGACCGUAAAGCUCUGACGCCCUGUCCUAGGACAGGGCGUCAUCUUUUUUGUC -s -1 -c 1 -r GGGATGATTTACTAATAGCTCTTGGTATGGTATAAT -v -d
# LEAPHKCP_01380 
get_ID_info LEAPHKCP_01380 -t AUUAUUUGCGUUUUUGUUGGAAAGGAAGCCGAUAAUAAUCGAAACGAAAGUUUUCCGAGUGGGGCAACGGCGAUCCGGUCUUGUAUCGGGGUAGCCGUCCCCUCAGAAGAAAAGGGGGCUGACCGCGCGUCACCCCCUUUUCAUUUUGCCG -s -1 -c -1 -r GTTATCGCTTCCTTTCCAACAAAAATGCGATATAAT -v -d